In [19]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture
from scipy.special import kl_div
import sys

In [20]:
print(sys.executable)

/usr/bin/python3


In [3]:
d = np.load("drors_for_all.npy")
X = d[0:-1:10,:4]
Y = d[0:-1:10,4]

In [17]:


class PipelineGrid:

    def __init__(self, dim_reducer, param_grid, classifier):
        self.dim_reducer = dim_reducer
        self.classifier = classifier
        self.param_grid = param_grid
        self.pipeline = Pipeline(steps=[("reducer", dim_reducer), ("classifier", classifier)])
        self.grid_search = GridSearchCV(self.pipeline, param_grid=param_grid)

    def fit(self, X): self.grid_search.fit(X)

    def get_best_estimator(self, X):
        self.fit(X)
        return self.grid_search.best_estimator_

    def get_all_estimators(self): return self.grid_search.cv_results_["params"]

    def make_subpipes(self, X):
        self.fit(X)
        pipes = []
        perms = self.get_all_estimators()
        for perm in perms:
            subpipe = Pipeline(steps=[("reducer", self.dim_reducer), ("classifier", self.classifier)])
            for k, v in perm.items():
                subpipe.set_params(**{k: v})
            pipes.append(subpipe)
        return pipes

    def get_populations(self, pipe, X):
        preds = pipe.fit_predict(X)
        populations = [preds[preds == i].shape[0] / preds.shape[0] for i in set(preds)]
        return populations

    def get_populations_of_all(self, X, Y):
        subpipes = self.make_subpipes(X)
        pipe_data = {}
        for pipe in subpipes:
            populations = {}
            for system in set(Y):
                system_data = X[np.where(Y == system),:]
                populations[system] = self.get_populations(pipe, system_data)
            pipe_data[pipe] = populations
        return pipe_data



In [18]:
dr = PCA()
c = GaussianMixture()
params = {"reducer__n_components": [1,2,3], "classifier__n_components": [2,3,4,5]}

pipe = PipelineGrid(dim_reducer=dr, param_grid=params, classifier=c)

print(pipe.get_populations_of_all(X, Y))

ValueError: Found array with dim 3. Estimator expected <= 2.